In [ ]:
%pip install --upgrade --quiet  lark qdrant-client

In [7]:
from langchain_community.vectorstores import Qdrant
from langchain_core.documents import Document
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

from langchain.document_loaders import TextLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.docstore.document import Document
from typing import List, Dict
from langchain.chains import RetrievalQA
import os

HF_EMBEDDING = HuggingFaceEmbeddings(model_name="dangvantuan/vietnamese-embedding")
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1,api_key= os.getenv('llm_api_1'))


# Function to load and chunk data from a folder
def load_and_chunk_data(data_path):
    docs = []
    # Load all .txt files from the specified folder
    for filename in os.listdir(data_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(data_path, filename)
            loader = TextLoader(file_path, encoding='utf-8')
            docs.extend(loader.load())

    # Define headers to split on for Markdown splitting
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    # Initialize Markdown splitter
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=True
    )

    chunk_size = 512
    chunk_overlap = 0

    # Initialize character-based splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    chunked_docs = []

    # Process each document
    for doc in docs:
        # Split document by Markdown headers
        md_header_splits = markdown_splitter.split_text(doc.page_content)

        # Further split the Markdown header splits into smaller chunks
        chunked_docs.extend(text_splitter.split_documents(md_header_splits))

    return chunked_docs


data_path = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/data/thongtintuyensinh'
chunked_data = load_and_chunk_data(data_path)

from langchain_chroma import Chroma
vectordb = Chroma.from_documents(documents=chunked_data, embedding=HF_EMBEDDING)

# vectorstore = Qdrant.from_documents(
#     chunked_data,
#     HF_EMBEDDING,
#     location=":memory:",  # Local mode with in-memory storage only
#     collection_name="test1",
# )

/home/justtuananh/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "năm 2024 học viện lấy bao nhiêu nữ"
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [10]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

8

In [16]:
from typing import List

from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

    # template="""You are an AI language model assistant. Your task is to generate five 
    # different versions of the given user question to retrieve relevant documents from a vector 
    # database. By generating multiple perspectives on the user question, your goal is to help
    # the user overcome some of the limitations of the distance-based similarity search. 
    # Provide these alternative questions separated by newlines. feedback in vietnamese
    # Original question: {question}""",

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
template="""Bạn là một trợ lý AI ngôn ngữ. Nhiệm vụ của bạn là tạo ra năm phiên bản khác nhau của câu hỏi do người dùng cung cấp để truy xuất các tài liệu liên quan từ cơ sở dữ liệu vector. 
Bằng cách tạo ra nhiều góc nhìn khác nhau về câu hỏi của người dùng, mục tiêu của bạn là giúp người dùng vượt qua một số hạn chế của tìm kiếm tương tự dựa trên khoảng cách. Hãy cung cấp các câu hỏi thay thế này, mỗi câu trên một dòng mới. 
Yêu cầu: Các câu hỏi phải được tạo ra bằng tiếng Việt.
Câu hỏi gốc: {question}"""
,
)


# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
question = "Năm 2023 học viện lấy bao nhiêu nữ"

In [18]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.invoke(input="năm 2023 học viện lấy bao nhiêu nữ")
len(unique_docs)

OutputParserException: Invalid json output: Here are five alternative versions of the original question:

Năm 2023, số lượng nữ sinh được nhận vào học viện là bao nhiêu?

Tỷ lệ nữ sinh trong học viện năm 2023 là bao nhiêu phần trăm?

Năm 2023, học viện có bao nhiêu chỉ tiêu cho nữ sinh?

Số lượng nữ sinh được tuyển vào học viện năm 2023 là bao nhiêu?

Năm 2023, học viện có bao nhiêu nữ sinh được nhận vào học?

In [12]:
import logging 

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [9]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})
results = retriever.invoke("năm 2024 học viện lấy bao nhiêu chỉ tiêu")
for result in results: 
    print(result.page_content)

Thực hiện theo Quyết định của Bộ Quốc phòng giao. Chỉ tiêu tuyển sinh được xác định theo giới tính (thí sinh nam, thí sinh nữ) và vùng, miền (miền Bắc được tính từ Quảng Bình trở ra, miền Nam được tính từ Quảng Trị trở vào). Chỉ tiêu tuyển sinh năm 2024 sẽ được Học viện thông báo chi tiết sau khi Bộ Quốc phòng giao chỉ tiêu chính thức. Các đối tượng tuyển sinh gồm:
Đối tượng tuyển sinh: Đào tạo kỹ sư quân sự (Chỉ huy, quản lý kỹ thuật)
Mã trường: KQH
Mã ngành: 7860220
Chỉ tiêu: 540
Một là, Học viện là môi trường tốt để các em phát huy được năng lực bản thân, được học tập để khám phá và làm chủ các công nghệ kỹ thuật tiên tiến được ứng dụng trong quân sự như: Trí tuệ nhân tạo, khoa học máy tính, công nghệ rô bốt, điện tử-viễn thông, kỹ thuật điều khiển tự động và tự động hoá, kỹ thuật ra đa-dẫn đường, kỹ thuật cơ khí động lực, kỹ thuật xây dựng, công nghệ hoá học, môi trường …
- Phương thức 5: Xét tuyển theo kết quả của kỳ thi tốt nghiệp THPT năm 2024. Thực hiện xét cho số chỉ tiêu còn 

### Retrieval

In [16]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
# add self-query
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
    AttributeInfo)
# Cấu hình thông tin metadata
metadata_field_info = [
    AttributeInfo(
        name="Header 1",
        description="""
Đây là tiêu đề cấp cao nhất, thường được sử dụng để giới thiệu tổng quan về chủ đề chính. 
Khi lọc dữ liệu, nếu truy vấn chứa lỗi chính tả hoặc cách viết khác của tên này, bạn nên lọc dựa trên giá trị của trường này thay vì dựa vào tên trong truy vấn.
 Phải kết hợp bộ lọc tiêu đề cấp 1 với các bộ lọc phụ đề sử dụng toán tử AND. 
 Luôn sử dụng một hoặc nhiều bộ lọc  và sử dụng toán tử OR để kiểm tra tất cả các trường khác. 
 Nếu giá trị của trường này chứa từ hoặc cụm từ tương tự như trong truy vấn, hãy lọc theo chuỗi chính xác từ giá trị thay vì từ truy vấn.
""",
        type="string",
    ),
    AttributeInfo(
        name="Header 2",
        description="""
Đây là tiêu đề cấp hai, được sử dụng để chia nhỏ nội dung thành các phần nhỏ hơn. 
Luôn luôn sử dụng một hoặc nhiều bộ lọc và sử dụng toán tử OR để kiểm tra tất cả các trường khác.
Nếu giá trị của trường này chứa từ hoặc cụm từ tương tự như trong truy vấn, hãy lọc theo chuỗi chính xác từ giá trị thay vì từ truy vấn.
Không bao giờ lọc trường này dựa trên giá trị của trường 'Header 1'.
""",
        type="string",
    ),
    AttributeInfo(
        name="Header 3",
        description="""
Đây là tiêu đề cấp ba, cung cấp thông tin chi tiết hơn. 
Luôn luôn sử dụng một hoặc nhiều bộ lọc  và sử dụng toán tử OR để kiểm tra tất cả các trường khác. 
Nếu giá trị của trường này chứa từ hoặc cụm từ tương tự như trong truy vấn, hãy lọc theo chuỗi chính xác từ giá trị thay vì từ truy vấn. 
Không bao giờ lọc trường này dựa trên giá trị của trường 'Header 1'
""",
        type="string",
    )
]


document_content_description = "Tài liệu nói về thông tin tuyển sinh của học viện kỹ thuật quân sự trong năm nay và các năm gần đây."

# Tạo SelfQueryRetriever
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True, enable_limit=True,
)


In [ ]:
examples = [
    (
        " Chỉ tiêu tuyển sinh của học viện năm 2024",
        {"query": "chỉ tiêu tuyển sinh, học viên năm 2024",
         "filter": 'and(like("Header 1", "THÔNG TIN KỲ TUYỂN SINH CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024"), or(like("Header 2", "Chỉ tiêu tuyển sinh"), '
                   'like("Header 3", "Chỉ tiêu tuyển sinh")))'}
    ),
    (
        "Các phương thức xét tuyển tại học viện kỹ thuật quân sự",
        {"query": "phương thức xét tuyển, học viện kỹ thuật quân sự",
         "filter": 'and(like("Header 1", "THÔNG TIN KỲ TUYỂN SINH CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024"), or(like("Header 2", "Tổ chức xét tuyển"), '
                   'like("Header 3", "Phương thức 1: Xét tuyển thẳng**")))'}
    ),
    (
        "Điểm trúng tuyển năm 2023 của học viện",
        {"query": "trúng tuyển, 2023",
         "filter": 'and(like("Header 1", "Thông tin tuyển sinh năm 2021, 2022, 2023 của học viện kỹ thuật quân sự"), or(like("Header 2", "4.2. Điểm trúng tuyển năm 2021, 2022, 2023**"), '
                   'like("Header 3", "4.2. Điểm trúng tuyển năm 2021, 2022, 2023**")))'}
    )
]

In [ ]:
examples = [
    (
        "Admission quota of the academy for 2024",
        {
            "query": "admission quota, academy, 2024",
            "filter": 'and(like("Header 1", "ADMISSION INFORMATION OF MILITARY TECHNICAL ACADEMY IN 2024"), '
                      'or(like("Header 2", "Admission quota"), like("Header 3", "Admission quota")))'
        }
    ),
    (
        "Admission methods at the Military Technical Academy",
        {
            "query": "admission methods, military technical academy",
            "filter": 'and(like("Header 1", "ADMISSION INFORMATION OF MILITARY TECHNICAL ACADEMY IN 2024"), '
                      'or(like("Header 2", "Admission process"), like("Header 3", "Method 1: Direct admission**")))'
        }
    ),
    (
        "Admission scores for 2023 of the academy",
        {
            "query": "admission scores, 2023",
            "filter": 'and(like("Header 1", "Admission information of Military Technical Academy for 2021, 2022, 2023"), '
                      'or(like("Header 2", "4.2. Admission scores for 2021, 2022, 2023**"), '
                      'like("Header 3", "4.2. Admission scores for 2021, 2022, 2023**")))'
        }
    )
]


In [17]:
prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info
)
query = "Tôi vào học viện rôi thì phải đóng bao nhiêu tiền học phí vậy?"
print(prompt.format(query="{query}"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [18]:
chain = load_query_constructor_runnable(
    llm=llm,
    attribute_info=metadata_field_info,
    document_contents=document_content_description,
    fix_invalid=True
)

In [25]:
query = "Điểm trúng tuyển cho thí sinh nữ có thường trú phía Nam năm 2021 là bao nhiêu?"
chain.invoke(({"query": query}))

StructuredQuery(query='điểm trúng tuyển nữ phía nam 2021', filter=None, limit=None)

In [26]:
from langchain.retrievers import SelfQueryRetriever

retriever = SelfQueryRetriever(
    query_constructor=chain, vectorstore=vectorstore, verbose=True
)

In [27]:
sq_retriever = SelfQueryRetriever(
    query_constructor=chain,
    vectorstore=vectorstore,
    verbose=True,
)
sq_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=sq_retriever,
    return_source_documents=True
)

In [28]:
sq_retriever = SelfQueryRetriever(
    query_constructor=chain,
    vectorstore=vectorstore,
    verbose=True,
)

In [34]:
# không add thêm ví dụ vào 
sq_qa.invoke("Thí sinh nam có thường trú phía Bắc cần đạt bao nhiêu điểm để trúng tuyển vào Học viện Kỹ thuật Quân sự năm 2022?")

{'query': 'Thí sinh nam có thường trú phía Bắc cần đạt bao nhiêu điểm để trúng tuyển vào Học viện Kỹ thuật Quân sự năm 2022?',
 'result': "I don't know. The provided context does not mention the specific points required for admission to the Military Technical Academy in 2022. It only mentions the admission methods and criteria, but not the specific scores or points required for admission.",
 'source_documents': [Document(metadata={'Header 1': 'CƠ HỘI HỌC TẬP VÀ ĐIỂM MỚI TUYỂN SINH NĂM 2024 CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ', 'Header 2': 'II. MỘT SỐ ĐIỂM MỚI TRONG CÔNG TÁC TUYỂN SINH TẠI HỌC VIỆN KỸ THUẬT QUÂN SỰ NĂM 2024', '_id': 'e962eaf6d48b4c279d3d00267fd3d6a2', '_collection_name': 'tintuyensinh'}, page_content='4. Xét tuyển dựa vào kết quả thi tốt nghiệp THPT theo quy chế của Bộ Giáo dục và Đào tạo.\nNhư vậy, so với năm 2023, trong kỳ tuyển sinh đại học năm 2024 Học viện Kỹ thuật quân sựbổ sung phương thức lấy kết quả thi đánh giá năng lực của Đại học Quốc gia Hà Nội và Đại học Quốc gi